In [7]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
import time
import csv
import psycopg2
from sqlalchemy import create_engine
import schedule

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
#Función para recopilar data en tiempo real sobre las estaciones de bicimad y poder almacenarla en una database
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    stations_real_time = pd.DataFrame(stations["data"])
    stations_real_time["date_time"] = date_and_time_formated
    stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    stations_real_time = stations_real_time.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
    stations_real_time.to_csv(f'../data_csv/stations_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    stations_real_time.to_sql('stations', con=engine, if_exists='append', index=False)
    return stations_real_time

In [6]:
get_stations()

,address,code_district,code_suburb,dock_bikes,free_bases,id,light,name,no_available,number,reservations_count,total_bases,date_time,longitude,latitude,coordinates
0,"Calle María de las Mercedes de Borbón, 214,",16,166,11,12,2365,2,"262 - María de las Mercedes de Borbón, 214",0,262,0,23,2024-02-09 11:57:42,-3.620114,40.492868,"(-3.620114, 40.4928683)"
1,"Calle María de las Mercedes de Borbón, 98,",16,166,5,18,2366,0,"611 - María de las Mercedes de Borbón, 98",0,611,0,23,2024-02-09 11:57:42,-3.612107,40.490908,"(-3.6121068531215528, 40.490908297996654)"
2,"237- calle Francisco Balseiro,Comunidad de Mad...",06,061,13,11,2362,2,237 - calle Francisco Balseiro,0,237,0,24,2024-02-09 11:57:42,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
3,"para borrar,",16,166,0,0,1404,3,0 - para borrar,1,0,0,0,2024-02-09 11:57:42,-3.620180,40.492910,"(-3.62018, 40.49291)"
4,"Calle Miguel Moya nº 1,",01,015,19,8,1406,1,2 - Miguel Moya,0,2,0,27,2024-02-09 11:57:42,-3.705690,40.420400,"(-3.70569, 40.4204)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,"Av Abrantes, 55,",11,117,8,10,2153,2,"377 - Av Abrantes, 55",0,377,0,18,2024-02-09 11:57:42,-3.727836,40.380918,"(-3.72783615, 40.38091762)"
608,"Calle de Mota del Cuervo, 70,",16,163,12,12,2154,2,"539 - Calle de Mota del Cuervo, 70",0,539,0,24,2024-02-09 11:57:42,-3.631357,40.464876,"(-3.63135715, 40.4648762)"
609,"Calle Seseña, 93,",10,104,17,7,2155,1,"334 - Seseña, 93",0,334,0,24,2024-02-09 11:57:42,-3.768156,40.394973,"(-3.76815631, 40.39497294)"
610,"Calle Cardaño, 10,",10,105,10,14,2156,2,"335 - Calle Cardaño, 10",0,335,0,24,2024-02-09 11:57:42,-3.770575,40.396598,"(-3.77057541, 40.39659757)"


In [8]:
#Esto hace que se ejecute la función cada hora y se carguen los datos en mi base de datos
schedule.every().hour.do(get_stations)

while True:
    schedule.run_pending()
    time.sleep(1)